### Tecnolgías del Lenguaje. Entregable 4
---
#### Reducción basada en similitud

Se aplica un **umbral de similitud** para asegurar que solo se conserven aquellos posts que realmente están relacionados con el rasgo psicológico que se está analizando. Las similitudes bajas o negativas indican que el contenido del post no guarda relación significativa con el vector representante del rasgo, por lo que incluirlos introduciría ruido y debilitaría la interpretación. Al mantener únicamente los posts con una similitud superior a 0.3, se garantiza que el conjunto final esté compuesto por ejemplos más coherentes, relevantes y explicativos, lo que permite obtener conclusiones más fiables sobre cómo se manifiesta cada rasgo en el lenguaje de los usuarios. Después, comprueba que por cada combinación de usuario, rasgo y polaridad existe al menos un post y si no quedó ninguno tras el filtro, rescata del archivo original el de mayor similitud, asegurando que nunca se pierde ese caso.

El resultado es un archivo reducido e informativo, almacenado en `material/top50_posts_per_user_reduced.csv`

In [11]:
import pandas as pd
import csv

# Cargar archivo original
df = pd.read_csv("material/top50_posts_per_user_MPNet_by_trait.csv")

THRESHOLD = 0.4

# 1️⃣ Filtrar por similarity > 0.4
df_filtrado = df[df["similarity"] > THRESHOLD].copy()

# 2️⃣ Eliminar casos donde el MISMO post aparece como positivo y negativo
# Nos quedamos con la fila de mayor similarity para cada usuario–trait–post
df_sin_conflictos = (
    df_filtrado
    .sort_values("similarity", ascending=False)
    .drop_duplicates(subset=["username", "trait", "post"], keep="first")
)

# 3️⃣ Garantizar mínimo 1 post por usuario–trait–polarity
def ensure_minimum_posts(group):
    if len(group) > 0:
        return group
    else:
        # Recuperar todos los posts originales de ese usuario–rasgo–polaridad
        original_group = df.loc[
            (df["username"] == group.name[0]) &
            (df["trait"] == group.name[1])
        ]
        # Seleccionar el de mayor similitud
        return original_group.nlargest(1, "similarity")

df_garantizado = (
    df_sin_conflictos
    .groupby(["username", "trait"], group_keys=False)
    .apply(ensure_minimum_posts)
)

# 4️⃣ Orden personalizado final
trait_order = [
    "Agreeableness",
    "Openness",
    "Conscientiousness",
    "Extraversion",
    "Neuroticism"
]

df_garantizado["trait"] = pd.Categorical(df_garantizado["trait"], categories=trait_order, ordered=True)

df_ordenado = df_garantizado.sort_values(
    by=["username", "trait", "similarity"],
    ascending=[True, True, False]
)

# Guardar archivo final
df_ordenado.to_csv(
    "material/top50_posts_per_user_MPNet_by_trait_reduced_clean.csv",
    index=False,
    quoting=csv.QUOTE_ALL
)

print("Filtrado completado. Posts finales:", len(df_garantizado))


C:\Users\Usuario\AppData\Local\Temp\ipykernel_3260\2717934238.py:36: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(ensure_minimum_posts)


Filtrado completado. Posts finales: 38525
